# Unsupervised Machine Learning for the Classification of Astrophysical X-ray Sources
###### *Víctor Samuel Pérez Díaz<sup>1,2</sup>, Rafael Martinez-Galarza<sup>1</sup>, Alexander Caicedo-Dorado<sup>2</sup>, Raffaele D'Abrusco<sup>1</sup>*

*1. Center for Astrophysics | Harvard & Smithsonian, 2. Universidad del Rosario*

Contact ```vperezdiaz@cfa.harvard.edu``` for questions or comments.


#### Clustering

In this notebook we preprocess the data and then cluster it using Gaussian Mixture Models. You can find here also validations for the number of clusters.

---

In [ ]:
%load_ext autoreload
%autoreload 2

from umlcaxs_lib import votable_to_pandas, lognorm, compute_bics, compute_silhouettes
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.mixture import GaussianMixture
from astropy.coordinates import SkyCoord
import astropy.units as u

from scipy.stats import gaussian_kde

%matplotlib inline

In [ ]:
# Edit the font, font size, and axes width

mpl.rcParams['font.family'] = 'Avenir LT Std'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2

#### Data preprocessing
---

In [ ]:
data = votable_to_pandas("../../data/cscresults.vot")

In [ ]:
data.shape

In [ ]:
# We define new measurement columns
data['var_ratio_b'] = data['var_sigma_b']/data['var_mean_b']
data['var_ratio_h'] = data['var_sigma_h']/data['var_mean_h']
data['var_ratio_s'] = data['var_sigma_s']/data['var_mean_s']

data['var_newq_b'] = ((data['var_max_b'] + data['var_min_b'])/2)/data['var_mean_b']

In [ ]:
# Features that we use in our analysis
features = ['hard_hm', 'hard_hs', 'hard_ms', 'powlaw_gamma', 'bb_kt', 'var_prob_b','var_ratio_b', 'var_prob_h', 'var_ratio_h', 'var_prob_s', 'var_ratio_s', 'var_newq_b']

# Features to log transform and normalize
features_lognorm = ['bb_kt', 'var_ratio_h', 'var_ratio_b', 'var_ratio_s', 'var_newq_b']

# Features to normalize
features_norm = ['powlaw_gamma']

# Drop data with missing values in features
X_df_out = data.dropna(subset=features)
X_df = X_df_out[features]

In [ ]:
print('Number of properties:', len(features))

In [ ]:
# Normalize or log normalize
X_df = lognorm(X_df, features, features_norm, features_lognorm)
X = X_df.to_numpy()

#### BIC and Silhouette plots
---

In [ ]:
bics_df = pd.read_csv('./model_scores/bics_df.csv', index_col=0)
bics_df.drop(np.arange(10), inplace=True)

silhouette_scores_df = pd.read_csv('./model_scores/silhouette_scores_df.csv', index_col=0)

bics_df_grad = bics_df.copy(deep=True)
bics_df_grad['grad'] = bics_df.groupby(['i'])['bic'].transform(np.gradient)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,10))
# make a little extra space between the subplots
fig.subplots_adjust(hspace=0.2)

ax1.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
ax1.yaxis.set_major_locator(mpl.ticker.MultipleLocator(100000))

ax2.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
ax2.yaxis.set_major_locator(mpl.ticker.MultipleLocator(40000))

sns.lineplot(data=bics_df, x='k', y='bic', color='black', ci='sd', markers=True, marker='o', ax=ax1)
ax1.set_ylabel('BIC')
ax1.set_xlim(2, 20)
ax1.set_xticks([2,6,10,15,20])
ax1.grid(True)
ax1.axvline(6, color='red', linestyle='--')
ax1.set_xlabel('')

sns.lineplot(data=bics_df_grad, x='k', y='grad', color='black', ci='sd', markers=True, marker='o', ax=ax2)
ax2.set_ylabel('grad BIC')
ax2.set_xlabel('Values of $K$')
ax2.set_xlim(2, 20)
ax2.set_xticks([2,6,10,15,20])
ax2.grid(True)
ax2.axvline(6, color='red', linestyle='--')
ax2.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.savefig('./figures/methods_1.pdf', bbox_inches='tight')

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(10,5))
# make a little extra space between the subplots
fig.subplots_adjust(hspace=0.2)

ax1.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
ax1.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.05))

sns.lineplot(data=silhouette_scores_df, x='k', y='silhouette', color='black', ci='sd', markers=True, marker='o', ax=ax1)
ax1.set_ylabel('Silhouette Score')
ax1.set_xlim(2, 20)
ax1.set_xticks([2,6,10,15,20])
ax1.grid(True)
ax1.axvline(6, color='red', linestyle='--')
ax1.set_xlabel('Values of $K$')

plt.savefig('./figures/methods_2.pdf', bbox_inches='tight')

#### GMM
---

In [ ]:
X_df_out_final = pd.read_csv('out_data/cluster_csc.csv', index_col=0)

In [ ]:
# Convert RA and Dec to galactic coordinates
coords = SkyCoord(X_df_out_final['ra'], X_df_out_final['dec'], unit=(u.deg, u.deg), frame='icrs')
galactic_coords = coords.galactic

# Add galactic coordinates to the DataFrame
X_df_out_final['l'] = galactic_coords.l.deg
X_df_out_final['b'] = galactic_coords.b.deg

# Define the range of galactic latitudes considered as the galactic plane
Y = 5

# Find the percentage of detections in the galactic plane for each cluster
clusters = X_df_out_final['cluster'].unique()
for cluster in clusters:
    cluster_data = X_df_out_final[X_df_out_final['cluster'] == cluster]
    in_galactic_plane = cluster_data[np.abs(cluster_data['b']) < Y]
    percentage = (len(in_galactic_plane) / len(cluster_data)) * 100
    print(f"Cluster {cluster}: {percentage:.2f}% of detections are at galactic latitudes smaller than {Y} degrees")

#### GMM plots
---

In [ ]:
markers = {0: "o", 1: "X", 2:"P", 3:"v", 4:"s", 5:"^"}
colors = {0: "Blues", 1: "Oranges", 2:"Greens", 3:"Reds", 4:"Purples", 5:"YlOrBr"}

In [ ]:
xarr = X_df_out_final.ra
yarr = X_df_out_final.dec
eq = SkyCoord(xarr[:], yarr[:], unit=u.deg)
gal = eq.galactic

X_df_out_final['l'] = gal.l.wrap_at('180d').radian
X_df_out_final['b'] = gal.b.radian

In [ ]:
from astroquery.simbad import Simbad
fig = plt.figure(figsize=(8,6))
ax = plt.subplot(111, projection='mollweide')
ax.grid(True)

scatter = sns.scatterplot(data=X_df_out_final, x='l', y='b', hue='cluster', palette="bright", style='cluster', markers=markers, rasterized=True, ax=ax)

plt.show()
#plt.savefig('./figures/results_1_anno.pdf', bbox_inches='tight')  

In [ ]:
df_coords = pd.DataFrame(np.concatenate([X_df_out_final.name.values.reshape(-1,1), gal.l.wrap_at('180d').radian.reshape(-1,1), gal.b.radian.reshape(-1,1), X_df_out_final.cluster.to_numpy().reshape(-1,1)], axis=1), columns=['name', 'ra', 'dec', 'cl'])
#df_coords.drop_duplicates('name', inplace=True)
df_coords['cl'] = df_coords.cl.astype('int')
df_coords['ra'] = df_coords.ra.astype('double')
df_coords['dec'] = df_coords.dec.astype('double')

In [ ]:
fig, axs = plt.subplots(2, 3, subplot_kw=dict(projection='mollweide'), figsize=(20,10), tight_layout=True)

for i, ax in enumerate(fig.axes):
    ax.set_title(f'Cluster {i}', size=16)
    
    df_coords_cl = df_coords[df_coords.cl == i]

    x=df_coords_cl.ra
    y=df_coords_cl.dec

    k = gaussian_kde(np.vstack([x, y]))
    xi, yi = np.mgrid[x.min():x.max():x.size**0.5*1j,y.min():y.max():y.size**0.5*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    cf = ax.pcolormesh(xi, yi, zi.reshape(xi.shape), alpha=1, shading='auto', cmap=colors[i], edgecolors='none', rasterized=True)

    # Contour option
    #levels = np.linspace(0, zi.max(), 25)
    #cf = ax.contourf(xi, yi, zi.reshape(xi.shape), levels=levels, alpha=1, cmap='Purples', rasterized=True)

    #ax.scatter(data=df_coords_cl[['ra', 'dec']], x='ra', y = 'dec', s=0.1, color='black', rasterized=True)

plt.savefig('./figures/results_2.pdf', bbox_inches='tight')  

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(20,10), tight_layout=True)

for i, ax in enumerate(fig.axes):
    ax.set_title(f'Cluster {i}', size=16)
    dfprovs_melted_plot = dfprovs_melted[dfprovs.cl==i].reset_index().melt(id_vars='index')
    sns.lineplot(data=dfprovs_melted_plot, x='variable', y='value', ci='sd', ax=ax, color='black', markers=True, marker='o')
    ax.grid(True)
    if i>=3:
        ax.set_xlabel('Cluster')
    else:
        ax.set_xlabel('')
    ax.set_ylabel('Probability')

plt.savefig('./figures/results_a1.pdf', bbox_inches='tight')  

In [ ]:
colx = X_df.columns.get_loc("hard_hm")
coly = X_df.columns.get_loc("hard_hs")

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
ax.grid(True)
#ax.set_rasterized(True)
sns.scatterplot(data=X_df_out_final, x='hard_hm', y='hard_hs', hue='cluster', palette="bright", style='cluster', markers=markers, s=30, rasterized=True)

ax.set_xlabel(X_df.columns[colx]);
ax.set_ylabel(X_df.columns[coly]);

plt.savefig('./figures/results_4.pdf', bbox_inches='tight')  

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(20,10), tight_layout=True)

for i, ax in enumerate(fig.axes):
    ax.set_title(f'Cluster {i}', size=16)
    
    ax.grid(True)
    gs=sns.scatterplot(data=X_df_out_final[X_df_out_final.cluster==i], x='hard_hm', y='hard_hs', s=30, hue='var_prob_h', palette='viridis_r', ax=ax, rasterized=True)
    gs.legend_.remove()
    ax.set_xlabel(X_df.columns[colx])
    ax.set_ylabel(X_df.columns[coly])    
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-1.1,1.1])
    
cbar_ax = fig.add_axes([1, .3, .03, .4])
norm = plt.Normalize(X_df_out_final['var_prob_h'].min(), X_df_out_final['var_prob_h'].max())
sm = plt.cm.ScalarMappable(cmap="viridis_r", norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, label='var_prob_h')

plt.savefig('./figures/results_4a.pdf', bbox_inches='tight')  

In [ ]:
plt.style.use("dark_background")
fig, axs = plt.subplots(2, 3, figsize=(20,10), tight_layout=True)

for i, ax in enumerate(fig.axes):
    sns.set(style="ticks", context="talk")
    ax.set_title(f'Cluster {i}', size=16)
    
    #ax.grid(True)
    ax.set_xlabel(X_df.columns[colx])
    ax.set_ylabel(X_df.columns[coly])
    g=sns.kdeplot(data=X_df_out_final[X_df_out_final.cluster==i], x='hard_hm', y='hard_hs', cmap='inferno', fill=True, alpha=1, ax=ax)
    
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-1.1,1.1])

plt.savefig('./figures/results_4b.pdf', bbox_inches='tight')  

In [ ]:
def move_legend(ax, new_loc, **kws):
    '''https://github.com/mwaskom/seaborn/issues/2280'''
    old_legend = ax.legend_
    handles = old_legend.legendHandles
    labels = [t.get_text() for t in old_legend.get_texts()]
    title = old_legend.get_title().get_text()
    ax.legend(handles, labels, loc=new_loc, title=title, **kws)

def hist_plots(X_df_final_out, features, features_lognorm):
    
    nrow = 4; ncol = 3;
    fig, axs = plt.subplots(nrows=nrow, ncols=ncol, figsize=(16,20))
    fig.tight_layout(h_pad=10, w_pad=2)
    for i, ax in enumerate(axs.reshape(-1)): 
        if i >= len(features):
            ax.set_axis_off()
            continue
        ax.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='in')
        ax.yaxis.set_tick_params(which='minor', size=3, width=0.5, direction='in')
        ax.xaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
        ax.xaxis.set_tick_params(which='minor', size=3, width=0.5, direction='out')
        
        if features[i] in features_lognorm:
            X_desc = X_df_final_out[features[i]].copy(deep=True)
            nonzero = X_desc[X_desc!=0]
            minval = np.min(nonzero)/10

            X_df_final_out[(features[i]+'_log')] = X_desc + minval
            axsns = sns.histplot(data=X_df_final_out, x=(features[i]+'_log'), hue='cluster', ax=ax , palette='bright', bins=60, element="step", log_scale=True, stat='probability', common_norm=False)
            ax.set_xlabel('log({})'.format(features[i]))
        else:
            axsns = sns.histplot(data=X_df_final_out, x=features[i], hue='cluster', ax=ax , palette='bright', bins=60, element="step", stat='probability', common_norm=False)
        
        axsns.set(ylabel=None)
        move_legend(ax,
            new_loc="lower center",
            bbox_to_anchor=(.5, 1), ncol=3
        )

        plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
    
    plt.savefig('figures/results_5.pdf', dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
hist_plots(X_df_out_final, features, features_lognorm)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='in')
ax.yaxis.set_tick_params(which='minor', size=3, width=0.5, direction='in')
ax.xaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
ax.xaxis.set_tick_params(which='minor', size=3, width=0.5, direction='out')
axsns = sns.histplot(data=X_df_out_final, x='hard_hs', hue='cluster', ax=ax , palette='bright', bins=60, element="step", stat='probability', common_norm=False)

axsns.set(ylabel=None)
move_legend(ax,
new_loc="lower center",
bbox_to_anchor=(.5, 1), ncol=3
)

plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.savefig('figures/results_6a.pdf', dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='in')
ax.yaxis.set_tick_params(which='minor', size=3, width=0.5, direction='in')
ax.xaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
ax.xaxis.set_tick_params(which='minor', size=3, width=0.5, direction='out')
axsns = sns.histplot(data=X_df_out_final, x='var_prob_b', hue='cluster', ax=ax , palette='bright', bins=60, element="step", stat='probability', common_norm=False)

axsns.set(ylabel=None)
move_legend(ax,
new_loc="lower center",
bbox_to_anchor=(.5, 1), ncol=3
)

plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.savefig('figures/results_6b.pdf', dpi=300, transparent=False, bbox_inches='tight')